In [1]:
#%matplotlib notebook

# Import required packages
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.ticker as tic
import numpy as np
import pandas as pd


import sys
sys.path.append("..")
from codes import spectral_overlap_functions as sof # .py file with functions needed to quantify spectral overlap


# Retrieve stored dataframe
%store -r table1_hzm


<img src="../figures/ExoPhotlogo.png"   
     width=150px
     alt="ExoPhot icon"
     style="float: left; margin-right: 10px;" /> 

# ExoPhot spectral overlap interface
This notebook is intended for the selection of stars, exoplanetary atmospheres and photisystems and plotting their spectral absorption rate. It also calculates the total absorption rate and tabulates the values for the selected systems.



## Star selection
Select the spectral type of the host star:

In [2]:
stellar_type_list = ['a5v', 'g2v','m8v']

selected_stellar_type = widgets.Dropdown(
                options=stellar_type_list,
                value='g2v',
                description='Stellar type',
                disabled=False)


def stellar_type_handler(change):
     change.new  
    
selected_stellar_type.observe(stellar_type_handler, names='value')

display(selected_stellar_type)



Dropdown(description='Stellar type', index=1, options=('a5v', 'g2v', 'm8v'), value='g2v')

## Exoplanet selection

Select the inner, middle or outer part of the exoplanet habitable zone and the type of atmosphere (os1: Earth-like; os2: highly oxidizing, os3; weakly oxidizing; os4: reducing):


In [3]:
hz_list = ['hzi', 'hzm','hzo']
atmosphere_list = ['os1','os2','os3','os4']

selected_hz = widgets.Dropdown(
                options=hz_list,
                value='hzm',
                description='HZ',
                disabled=False)

selected_atmosphere = widgets.Dropdown(
                options=atmosphere_list,
                value='os1',
                description='Atmosphere type',
                disabled=False)

def hz_handler(change):
    change.new  
def atmosphere_handler(change):
    change.new  
    
selected_hz.observe(hz_handler, names='value')
selected_atmosphere.observe(atmosphere_handler, names='value')

display(selected_hz)
display(selected_atmosphere)



Dropdown(description='HZ', index=1, options=('hzi', 'hzm', 'hzo'), value='hzm')

Dropdown(description='Atmosphere type', options=('os1', 'os2', 'os3', 'os4'), value='os1')

## Pigment selection
Select one of the available pigments:

In [4]:
pigment_list = ['BChlA','BChlB','ChlA','ChlB','phot0']

selected_pigment = widgets.Dropdown(
                options=pigment_list,
                value='ChlA',
                description='Pigment',
                disabled=False)

def pigment_handler(change):
    change.new  
    
selected_pigment.observe(pigment_handler, names='value')

display(selected_pigment)



Dropdown(description='Pigment', index=2, options=('BChlA', 'BChlB', 'ChlA', 'ChlB', 'phot0'), value='ChlA')

# Table of results

In [5]:
current_system = table1_hzm.loc[(table1_hzm["Star"]==selected_stellar_type.value) & 
                      (table1_hzm["Atmosphere"]==selected_atmosphere.value) & 
                      (table1_hzm["HZ"]==selected_hz.value) & 
                      (table1_hzm["Pigment"]==selected_pigment.value)]

#Set precision of pandas to 2 
pd.set_option('precision', 2)

current_system[["Star","HZ","Atmosphere","Pigment","gamma_t","gamma_B","gamma_Q","ppfd","S"]].style


,Star,HZ,Atmosphere,Pigment,gamma_t,gamma_B,gamma_Q,ppfd,S
43,g2v,hzm,os1,ChlA,9.20,4.65,4.55,2115.69,1370.04


# Spectral Absorption Rate Calculation

In [6]:
def plot_absorption_rate():
    # specify file names
    output_file='./output/'+'atm_'+selected_stellar_type.value+'_'+selected_hz.value+'_'+selected_atmosphere.value+'_'+selected_pigment.value+'.txt'
    
    # read files to retrieve spectra
    data = np.loadtxt(output_file,usecols=(0,1,2,3,4)) 
    data = np.transpose(data)
    
    # reorder parameters for plotting purposeses
    y_sorted = [4,1,2,3]
    y_values = data[y_sorted]
    
    #Flux in Joules transformed to ergs 
    # 1J = 10E7 ergs
    y_values[1]=y_values[1]*1e7
    
    
    colors = ['black', 'red', 'blue', 'green'] #'#377eb8', '#ff7f00', '#4daf4a','#f781bf'
    labels = ['Spectral Absorption Rate ($\AA^{-1}$ s$^{-1}$)', 
              'Flux (erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)', 
              'Transmittance', 
              'Absorption Cross section (cm$^{2}$)']
    
    # define other paramters (e.g. linestyle etc.) in lists
    
    # plot of curve and solution
    fig_layout = Layout(width='720px', height='500px')
    fig, ax_orig = plt.figure(layout=fig_layout, title='Spectral Absorption Rate', display_toolbar=True,figsize=(8, 5))
    fig.pyplot = Toolbar(figure=fig)

    
    #fig, ax_orig = plt.subplots(figsize=(8, 5))
    for i, (arr, color, label) in enumerate(zip(y_values, colors, labels)):
        if i == 0:
            ax = ax_orig
            ax.set_ylabel(label, color=color)
            ax.plot(data[0],y_values[0], color=color, linestyle='-',linewidth=1, alpha=0.8)
        else:
            ax = ax_orig.twinx()
            ax.spines['right'].set_position(('outward', 50 * (i - 1)))
            ax.plot(data[0],arr, color=color, linestyle='-',linewidth=1, alpha=0.6)
            ax.set_ylabel(label, color=color)
            
            yfmt = tic.ScalarFormatter(useMathText=True)
            yfmt.set_powerlimits((-6, -5))
            ax.yaxis.set_major_formatter(yfmt)
            ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
            ax.get_yaxis().get_offset_text().set_visible(False)
            ax_max = max(ax.get_yticks())
            exponent_axis = np.floor(np.log10(ax_max)).astype(int)
            if exponent_axis!=0:
                ax.annotate(r'$\times$10$^{%i}$'%(exponent_axis),
                 xy=((0.999+(0.15 * (i - 1))), .999), color=color, xycoords='axes fraction')
    
            #ax.get_yaxis().get_offset_text().set_position(0.5, 0)
        ax.set_xlabel('Wavelength ($\AA$)')    
    
        ax.tick_params(axis='y', colors=color)
    fig.tight_layout()
    plt.savefig('visual_spectral_overlap_plot_PNAS_v3.pdf', dpi=150)
    plt.show()



In [7]:
from ipywidgets import *

import bqplot.pyplot as plt
from bqplot import Toolbar

#Input parameters layout
header  = Button(description="Input values",
                 layout=Layout(width='auto', grid_area='header'))

txt_layout = Layout(width='160px')

star_box = widgets.Dropdown(
                options=stellar_type_list,
                layout=txt_layout,
                value='g2v',
                description='Stellar type',
                disabled=False)

hz_box = widgets.Dropdown(
                options=hz_list,
                layout=txt_layout,
                value='hzm',
                description='HZ',
                disabled=False)

atmosphere_box = widgets.Dropdown(
                options=atmosphere_list,
                layout=txt_layout,
                value='os1',
                description='Atmosphere type',
                disabled=False)

pigment_box = widgets.Dropdown(
                options=pigment_list,
                layout=txt_layout,
                value='ChlA',
                description='Pigment',
                disabled=False)

go_btn = Button(description='UPDATE', button_style='success', layout=Layout(width='100px'))
reset_btn = Button(description='Reset', button_style='success', layout=Layout(width='100px'))

# Update and Reset buttons functions
def update():
    selected_pigment.observe(pigment_handler, names='value')
    plot_absorption_rate()

def reset():
    curve.scales['x'].min = .4
    curve.scales['x'].max = 2.5
    
    curve.scales['y'].min = -.5
    curve.scales['y'].max = .4
    sol_path.x = sol_path.y = []
    sol_points.x = sol_points.y = []
    sol_lbl.value = ''
    


#gamma_t_lbl_tmpl = 'x = {:.4f}'
gamma_t_lbl = selected_pigment

gamma_t_lbl.layout.width = '300px'


#call to Update and Reset functions
go_btn.on_click(lambda btn: update())
reset_btn.on_click(lambda btn: reset())

# Show the layout
final_fig = VBox([fig, fig.pyplot], 
                 layout=Layout(overflow_x='hidden'))
HBox([ VBox([header, star_box, hz_box, atmosphere_box, pigment_box, go_btn, reset_btn, gamma_t_lbl]),final_fig])

#current_system[["Star","HZ","Atmosphere","Pigment","gamma_t","gamma_B","gamma_Q","ppfd","S"]].style


NameError: name 'fig' is not defined